In [41]:
import keras
import io
import os
import sys
import numpy as np
import re
import csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import layers
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Conv2D, Bidirectional
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split

In [50]:
data_url = 'shakespeare.txt'
with io.open(data_url, 'r', encoding='utf8') as f:
    text = f.read()
vocab = set(text)
vocab_to_int = {c:i for i , c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
print(int_to_vocab)
train_data = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
label = train_data[1:]
train_data = train_data[:-1]

{0: 'd', 1: 'a', 2: 'S', 3: 'b', 4: '3', 5: 'N', 6: 'I', 7: 'V', 8: 'H', 9: 's', 10: '&', 11: 'B', 12: 'G', 13: 'r', 14: ',', 15: 'W', 16: 'F', 17: 'h', 18: 'v', 19: 'i', 20: 'n', 21: 'f', 22: ' ', 23: 'y', 24: "'", 25: 'D', 26: 'q', 27: 'R', 28: 'Y', 29: 'e', 30: 'j', 31: 'J', 32: 'L', 33: 'M', 34: 'Z', 35: '!', 36: 'X', 37: 'w', 38: 'p', 39: ':', 40: '?', 41: '.', 42: '-', 43: 'K', 44: 'C', 45: 'A', 46: 'U', 47: 'g', 48: 'P', 49: '$', 50: 'z', 51: 'Q', 52: 'k', 53: '\n', 54: 'm', 55: 'O', 56: 'T', 57: 'o', 58: 'E', 59: 'u', 60: 'c', 61: ';', 62: 'x', 63: 'l', 64: 't'}


In [51]:
maxlen = 40
chars = 65
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, 65)),
        layers.LSTM(128),
        layers.Dense(65, activation="softmax"),
    ]
)

In [47]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 600893
Total chars: 56
Number of sequences: 200285


In [49]:
print(maxlen)
print(len(chars))
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

40
56
